# Comparison of the reconstruction by SWoTTeD and fastSWoTTeD

In [1]:
# environnement setting
%pwd
import sys
sys.path.append("..")

In [2]:
import torch
from model.swotted import swottedModule, swottedTrainer, SlidingWindow
from model.fastswotted import fastSWoTTeDModule, SlidingWindowConv


In [3]:
swotted = SlidingWindow()
fastswotted = SlidingWindowConv()

In [5]:
Ph = torch.zeros( (2, 4, 3) ) # R=2 phenotypes, with K=4 event types and \omega=3 (duration)
Wp= torch.zeros( (2, 10) ) # R=2

# two simple and orthogonal phenotypes
Ph[0,:3,:3]=torch.eye( 3 )
Ph[1,3,:]=torch.ones( 3 )

# random assignement of nb phenotypes per type of phenotypes
nb=3
v=torch.randint(0, 10, (2,nb))
for d in range(2):
    for r in range(nb):
        Wp[d,v[d,r]]=1

In [6]:
Ph

tensor([[[1., 0., 0.],
         [0., 1., 0.],
         [0., 0., 1.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.],
         [1., 1., 1.]]])

In [7]:
Wp

tensor([[1., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 1.]])

In [8]:
swotted.reconstruct(Wp, Ph)

tensor([[1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1.]])

In [9]:
fastswotted.reconstruct(Wp, Ph.flip(2)) # we need to flip Ph ... because the model use flipped phenotypes!

tensor([[[1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1.]]])

## Comparison of the phenosuccession losses

In [10]:
from model.fastswotted import phenotypeNonSuccession_loss as fast_pns
from model.decomposition_contraints import phenotypeSuccession_constraint as pns

ImportError: cannot import name 'phenotypeNonSuccession_loss' from 'model.fastswotted' (/home/tguyet/Progs/swotted/notebooks/../model/fastswotted.py)

In [11]:
from math import log
def pns_base(Wk, Tw):
    penalisation = 0
    for p in range(len(Wk)):
        for r in range(Wk[p].shape[0]):
            for t in range(Wk[p].shape[1]):
                sum = 1
                for l in range(t - Tw, t + Tw + 1):
                    if l >= 0 and l < Wk[p].shape[1]:
                        sum += Wk[p][r][l].item()
                #if sum==0:
                #    sum = 1
                print(sum)
                penalisation += Wk[p][r][t].item() * log(sum)
    return penalisation

In [ ]:
pns(Wp.unsqueeze(dim=0), 3)

2.0794415416798357

In [ ]:
fast_pns(Wp.unsqueeze(dim=0), 3)

tensor(4.3944)

In [ ]:
pns_base(Wp.unsqueeze(dim=0), 3)

2.0
2.0
3.0
3.0
3.0
3.0
3.0
2.0
2.0
1.0
1.0
2.0
2.0
2.0
3.0
3.0
3.0
3.0


4.394449154672439

In [ ]:
Tw = 3
Wk = Wp.unsqueeze(dim=0)
O = torch.transpose(
    torch.stack([torch.eye(Wk.shape[1]) for i in range(2 * Tw + 1 )]), 0, 2
)

In [ ]:
O

tensor([[[1., 1., 1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1., 1., 1.]]])

In [ ]:
Wk

tensor([[[0., 0., 0., 1., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 1.]]])

In [ ]:
torch.conv1d(Wk, O, padding=Tw)

tensor([[[1., 1., 2., 2., 2., 2., 2., 1., 1.],
         [0., 0., 1., 1., 1., 2., 2., 2., 2.]]])

In [ ]:
1.0
2.0
2.0
3.0
3.0
3.0
3.0
2.0
2.0
1.0
1.0
1.0
2.0
2.0
2.0
3.0
3.0
3.0

3.0

### Comparison with random tensors

In [12]:
Ph = torch.rand( (2, 4, 3) ) # R=2 phenotypes, with K=4 event types and \omega=3 (duration)
Wp= torch.rand( (2,9) ) # R=2


In [13]:
swotted.reconstruct(Wp, Ph)

tensor([[0.8210, 1.3800, 2.6914, 2.1006, 2.0621, 1.6699, 1.8775, 2.1299, 2.0175,
         1.3659, 0.6285],
        [0.4587, 1.0660, 2.1632, 1.9104, 1.7513, 1.1155, 1.5354, 1.6241, 1.6849,
         1.3032, 0.5311],
        [1.1266, 1.8525, 2.5644, 1.8155, 1.5464, 1.6739, 1.7951, 2.1278, 1.7776,
         0.8685, 0.2459],
        [0.7709, 1.6869, 2.5771, 2.0682, 1.6862, 1.2612, 1.9257, 2.1243, 1.8576,
         1.3411, 0.2483]])

In [14]:
fastswotted.reconstruct(Wp, Ph.flip(2)) # we need to flip Ph ... because the model use flipped phenotypes!

tensor([[[0.8210, 1.3800, 2.6914, 2.1006, 2.0621, 1.6699, 1.8775, 2.1299,
          2.0175, 1.3659, 0.6285],
         [0.4587, 1.0660, 2.1632, 1.9104, 1.7513, 1.1155, 1.5354, 1.6241,
          1.6849, 1.3032, 0.5311],
         [1.1266, 1.8525, 2.5644, 1.8155, 1.5464, 1.6739, 1.7951, 2.1278,
          1.7776, 0.8685, 0.2459],
         [0.7709, 1.6869, 2.5771, 2.0682, 1.6862, 1.2612, 1.9257, 2.1243,
          1.8576, 1.3411, 0.2483]]])

In [15]:
fast_pns(Wp.unsqueeze(dim=0), 3)

NameError: name 'fast_pns' is not defined

In [16]:
pns_base(Wp.unsqueeze(dim=0), 3)

3.3005073070526123
3.7976690530776978
4.566150367259979
4.939386188983917
5.055931866168976
4.688624858856201
3.8076741695404053
3.7589850425720215
3.261823296546936
4.091213285923004
4.566627085208893
4.92310643196106
5.707332789897919
5.519496262073517
5.256077289581299
4.306308448314667
3.7503592371940613
3.2749454379081726


15.85719235221925

In [ ]:
pns(Wp.unsqueeze(dim=0), 3)

8.772329398656115